In [2]:
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 35.8 MB/s eta 0:00:00


# Разбиение по датасетам

In [3]:
import re
import emoji
import pandas as pd

def clean_text(text):
    text = str(text).lower()
    text = emoji.replace_emoji(text, replace=' ')
    text = re.sub(r'[!"№;%:?*()\-=\/\\|@#$^&{}\[\]\'.,~`\n\r\t]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_english(text):
    return bool(re.fullmatch(r"[A-Za-z\s]+", text))

def count_by_rating(df, name):
    print(f"\n{name}")
    print(df['rating'].value_counts().sort_index())

file_path_ru = "/content/drive/MyDrive/Датасеты/RuReviews.csv"
labels = {"positive": 5, "negative": 1, "neautral": 3}
data_ru = []

with open(file_path_ru, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        words = line.split()
        if words and words[-1] in labels:
            rating = labels[words[-1]]
            text = " ".join(words[:-1])
            data_ru.append([rating, text])

ru = pd.DataFrame(data_ru, columns=["rating", "text"])
ru["text"] = ru["text"].astype(str).apply(clean_text)
ru = ru[ru["text"].str.strip() != ""]
ru = ru[~ru["text"].apply(is_english)]
ru = ru[ru["text"].str.split().str.len() > 1]
counts = ru['rating'].value_counts().sort_index()

wb = pd.read_csv('/content/drive/MyDrive/Датасеты/WB.csv')
wb['text'] = wb['text'].astype(str).apply(clean_text)
wb = wb[~wb["text"].apply(is_english)]
wb = wb[wb["text"].str.split().str.len() > 2]
wb = wb.dropna(subset=['rating', 'text'])
counts = wb['rating'].value_counts().sort_index()

count_by_rating(ru, "RuReviews после 1 очистки")
count_by_rating(wb, "WB после 1 очистки")
###########################################################################################
test_pos1 = ru[ru["rating"] == 5].head(8000)
test_neu1 = ru[ru["rating"] == 3].head(8000)
test_neg1 = ru[ru["rating"] == 1].head(8000)

test_df1 = pd.concat([test_pos1, test_neu1, test_neg1], ignore_index=True)
test_df1 = test_df1.sample(frac=1, random_state=42).reset_index(drop=True)
test_df1.to_csv("/content/drive/MyDrive/Датасеты/RuReviews_test_after_learning.csv", index=False)
ru = ru.drop(test_pos1.index)
ru = ru.drop(test_neu1.index)
ru = ru.drop(test_neg1.index)
count_by_rating(ru, "RuReviews после 2 очистки")
###########################################################################################
test_pos_ru = ru[ru["rating"] == 5].head(3000)
test_neu_ru = ru[ru["rating"] == 3].head(3000)
test_neg_ru = ru[ru["rating"] == 1].head(3000)
test_pos_wb = wb[wb["rating"] == 5].head(5000)
test_neu_wb = wb[wb["rating"] == 3].head(5000)
test_neg_wb = wb[wb["rating"] == 1].head(5000)

test_df2 = pd.concat([
    test_pos_ru, test_neu_ru, test_neg_ru,
    test_pos_wb, test_neu_wb, test_neg_wb
], ignore_index=True)

test_df2 = test_df2.sample(frac=1, random_state=42).reset_index(drop=True)
test_df2.to_csv("/content/drive/MyDrive/Датасеты/RuWB_test_after_learning.csv", index=False)
wb = wb.drop(test_pos_wb.index)
wb = wb.drop(test_neu_wb.index)
wb = wb.drop(test_neg_wb.index)
ru = ru.drop(test_pos_ru.index)
ru = ru.drop(test_neu_ru.index)
ru = ru.drop(test_neg_ru.index)
count_by_rating(ru, "RuReviews после 3 очистки")
count_by_rating(wb, "WB после 2 очистки")
###########################################################################################
test_pos_ru = ru[ru["rating"] == 5].head(3500)
test_neu_ru = ru[ru["rating"] == 3].head(3500)
test_neg_ru = ru[ru["rating"] == 1].head(3500)
test_df3 = pd.concat([test_pos_ru, test_neu_ru, test_neg_ru], ignore_index=True)
test_df3 = test_df3.sample(frac=1, random_state=42).reset_index(drop=True)
test_df3.to_csv("/content/drive/MyDrive/Датасеты/RuReviews_test.csv", index=False)
ru = ru.drop(test_pos_ru.index)
ru = ru.drop(test_neu_ru.index)
ru = ru.drop(test_neg_ru.index)
count_by_rating(ru, "RuReviews после 4 очистки")
###########################################################################################
test_pos_wb = wb[wb["rating"] == 5].head(3500)
test_neu_wb = wb[wb["rating"] == 3].head(3500)
test_neg_wb = wb[wb["rating"] == 1].head(3500)
test_df4 = pd.concat([test_pos_wb, test_neu_wb, test_neg_wb], ignore_index=True)
test_df4 = test_df4.sample(frac=1, random_state=42).reset_index(drop=True)
test_df4.to_csv("/content/drive/MyDrive/Датасеты/WB_test.csv", index=False)
wb = wb.drop(test_pos_wb.index)
wb = wb.drop(test_neu_wb.index)
wb = wb.drop(test_neg_wb.index)
count_by_rating(ru, "WB после 3 очистки")
###########################################################################################
val_pos_ru = ru[ru["rating"] == 5].head(3500)
val_neu_ru = ru[ru["rating"] == 3].head(3500)
val_neg_ru = ru[ru["rating"] == 1].head(3500)
val_pos_wb = wb[wb["rating"] == 5].head(3500)
val_neu_wb = wb[wb["rating"] == 3].head(3500)
val_neg_wb = wb[wb["rating"] == 1].head(3500)

val_df5 = pd.concat([
    val_pos_ru, val_neu_ru, val_neg_ru,
    val_pos_wb, val_neu_wb, val_neg_wb
], ignore_index=True)

val_df5 = val_df5.sample(frac=1, random_state=42).reset_index(drop=True)
val_df5.to_csv("/content/drive/MyDrive/Датасеты/RuWB_validation.csv", index=False)
wb = wb.drop(val_pos_wb.index)
wb = wb.drop(val_neu_wb.index)
wb = wb.drop(val_neg_wb.index)
ru = ru.drop(val_pos_ru.index)
ru = ru.drop(val_neu_ru.index)
ru = ru.drop(val_neg_ru.index)
count_by_rating(ru, "RuReviews после 5 очистки")
count_by_rating(wb, "WB после 4 очистки")
###########################################################################################
train_pos_ru = ru[ru["rating"] == 5]
train_neu_ru = ru[ru["rating"] == 3]
train_neg_ru = ru[ru["rating"] == 1]

pos_deficit = 56000 - len(train_pos_ru)
neu_deficit = 86800 - len(train_neu_ru)
neg_deficit = 56000 - len(train_neg_ru)

train_pos_wb = wb[wb["rating"] == 5].head(pos_deficit)
train_neu_wb = wb[wb["rating"] == 3].head(neu_deficit)
train_neg_wb = wb[wb["rating"] == 1].head(neg_deficit)

train_df6 = pd.concat([
    train_pos_ru, train_neu_ru, train_neg_ru,
    train_pos_wb, train_neu_wb, train_neg_wb
], ignore_index=True)

train_df6 = train_df6.sample(frac=1, random_state=42).reset_index(drop=True)
train_df6.to_csv("/content/drive/MyDrive/Датасеты/RuWB_train.csv", index=False)
wb = wb.drop(train_pos_wb.index)
wb = wb.drop(train_neu_wb.index)
wb = wb.drop(train_neg_wb.index)
ru = ru.drop(train_pos_ru.index)
ru = ru.drop(train_neu_ru.index)
ru = ru.drop(train_neg_ru.index)
count_by_rating(ru, "RuReviews после 6 очистки")
count_by_rating(wb, "WB после 5 очистки")
###########################################################################################
print('\n Распределение отзывов по классам в получившихся датасетах \n')
count_by_rating(test_df1, "RuReviews_test_after_learning.csv")
count_by_rating(test_df2, "RuWB_test_after_learning.csv")
count_by_rating(test_df3, "RuReviews_test.csv")
count_by_rating(test_df4, "WB_test.csv")
count_by_rating(val_df5, "RuWB_validation.csv")
count_by_rating(train_df6, "RuWB_train.csv")


RuReviews после 1 очистки
rating
1    29426
3    29184
5    28874
Name: count, dtype: int64

WB после 1 очистки
rating
1    105000
3    106437
5     97334
Name: count, dtype: int64

RuReviews после 2 очистки
rating
1    21426
3    21184
5    20874
Name: count, dtype: int64

RuReviews после 3 очистки
rating
1    18426
3    18184
5    17874
Name: count, dtype: int64

WB после 2 очистки
rating
1    100000
3    101437
5     92334
Name: count, dtype: int64

RuReviews после 4 очистки
rating
1    14926
3    14684
5    14374
Name: count, dtype: int64

WB после 3 очистки
rating
1    14926
3    14684
5    14374
Name: count, dtype: int64

RuReviews после 5 очистки
rating
1    11426
3    11184
5    10874
Name: count, dtype: int64

WB после 4 очистки
rating
1    93000
3    94437
5    85334
Name: count, dtype: int64

RuReviews после 6 очистки
Series([], Name: count, dtype: int64)

WB после 5 очистки
rating
1    48426
3    18821
5    40208
Name: count, dtype: int64

 Распределение отзывов по классам

# Обучение

In [ ]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report, f1_score
from torch.utils.data import Dataset

class RuWBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding="max_length",
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[idx], dtype=torch.long),
        }

train_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_train.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuWB_validation.csv')
test_ru_df = pd.read_csv('/content/drive/MyDrive/Датасеты/RuReviews_test.csv')
test_wb_df = pd.read_csv('/content/drive/MyDrive/Датасеты/WB_test.csv')
label_mapping = {1: 0, 3: 1, 5: 2}
train_df['label'] = train_df['rating'].map(label_mapping)
val_df['label'] = val_df['rating'].map(label_mapping)
test_ru_df['label'] = test_ru_df['rating'].map(label_mapping)
test_wb_df['label'] = test_wb_df['rating'].map(label_mapping)
train_texts = train_df['text'].tolist()
y_train = train_df['label'].values
val_texts = val_df['text'].tolist()
y_val = val_df['label'].values
test_ru_texts = test_ru_df['text'].tolist()
y_ru_test = test_ru_df['label'].values
test_wb_texts = test_wb_df['text'].tolist()
y_wb_test = test_wb_df['label'].values

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruBert-base")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/ruBert-base", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_dataset = RuWBDataset(train_texts, y_train, tokenizer)
val_dataset = RuWBDataset(val_texts, y_val, tokenizer)
test_ru_dataset = RuWBDataset(test_ru_texts, y_ru_test, tokenizer)
test_wb_dataset = RuWBDataset(test_wb_texts, y_wb_test, tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    macro_f1 = f1_score(labels, predictions, average='macro')
    return {"macro_f1": macro_f1}

training_args = TrainingArguments(
    output_dir="./RuBERT_last",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir="./logs",
    load_best_model_at_end=True,
    report_to="none",
    metric_for_best_model="macro_f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("/content/drive/MyDrive/RuBERT_new")
tokenizer.save_pretrained("/content/drive/MyDrive/RuBERT_new")

val_predictions = trainer.predict(val_dataset)
y_val_pred = val_predictions.predictions.argmax(axis=1)
print(classification_report(y_val, y_val_pred))
print("F1 (Validation):", round(f1_score(y_val, y_val_pred, average='macro')*100, 2))

test_ru_predictions = trainer.predict(test_ru_dataset)
y_test_ru_pred = test_ru_predictions.predictions.argmax(axis=1)
print(classification_report(y_ru_test, y_test_ru_pred))
print("F1 (RuReviwes test):", round(f1_score(y_ru_test, y_test_ru_pred, average='macro')*100, 2))

test_wb_predictions = trainer.predict(test_wb_dataset)
y_test_wb_pred = test_wb_predictions.predictions.argmax(axis=1)
print(classification_report(y_wb_test, y_test_wb_pred))
print("F1 (WB test):", round(f1_score(y_wb_test, y_test_wb_pred, average='macro')*100, 2))
